This is an example of a simple CNN developed, trained and utilized

AI was used to help generate the codebase

Note: Make sure that the tensorflow package is installed in your device.

In [28]:
# Lib imports
import os
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, regularizers
import numpy as np

In [29]:
# DATASET DIRECTORY CONFIGURATION
# Download and unzip the dataset from Kaggle, set the directory paths accordingly.
train_dir = r"A:\CCS 248 - Artificial Neural Networks\coding\25-26\CS 3B\AXEL_JOHN_NUQUI\muffin-vs-chihuahua\test"
test_dir  = r"A:\CCS 248 - Artificial Neural Networks\coding\25-26\CS 3B\AXEL_JOHN_NUQUI\muffin-vs-chihuahua\test"

In [30]:
# IMAGE PARAMETERS
# Used to resize the input images, also will determine the input size of your input layer.
IMG_SIZE = (128, 128)
BATCH_SIZE = 32

In [31]:
# DATA PREPROCESSING & AUGMENTATION
# Optional but recommended for image processing tasks, especially with limited data.
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='training'
)
val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

Found 948 images belonging to 2 classes.
Found 236 images belonging to 2 classes.
Found 1184 images belonging to 2 classes.


In [32]:
# SIMPLE CNN MODEL ARCHITECTURE

# Some modifications are applied
initial_learning_rate = 0.001
# We are combining ExponentialDecay with Adam optimizer for better learning rate management
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

# Create the optimizer with the learning rate schedule
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

# Applied dropout layers to reduce overfitting
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3),
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(64, (3,3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.25),

    layers.Conv2D(128, (3,3), activation='relu',
                  kernel_regularizer=regularizers.l2(0.001)),
    layers.MaxPooling2D((2,2)),
    layers.Dropout(0.3),

    layers.Flatten(),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(0.5),

    layers.Dense(1, activation='sigmoid')
])

c:\Users\Axel John Nuqui\AppData\Local\Programs\Python\Python313\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [33]:
# Configure the model optimizers, loss function, and metrics
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) # old
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [34]:
# TRAINING THE CNN
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 14s 431ms/step - accuracy: 0.5222 - loss: 1.1274 - val_accuracy: 0.5424 - val_loss: 0.9725
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 13s 420ms/step - accuracy: 0.5844 - loss: 0.9193 - val_accuracy: 0.6271 - val_loss: 0.8309
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 12s 410ms/step - accuracy: 0.6667 - loss: 0.8210 - val_accuracy: 0.5975 - val_loss: 0.8481
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 12s 414ms/step - accuracy: 0.7247 - loss: 0.7330 - val_accuracy: 0.7585 - val_loss: 0.6640
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 13s 425ms/step - accuracy: 0.7584 - loss: 0.6767 - val_accuracy: 0.7712 - val_loss: 0.5912
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 13s 435ms/step - accuracy: 0.7690 - loss: 0.6576 - val_accuracy: 0.7246 - val_loss: 0.6398
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 13s 444ms/step - accuracy: 0.7954 - loss: 0.5910 - val_accuracy: 0.8136 - val_loss: 0.5808
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 12s 401ms/step - accuracy: 0.7901 - loss: 0.5970 - val_accu

In [35]:
# EVALUATE THE MODEL
test_loss, test_acc = model.evaluate(test_generator)
print(f"Test Accuracy: {test_acc}")

37/37 ━━━━━━━━━━━━━━━━━━━━ 6s 155ms/step - accuracy: 0.8133 - loss: 0.5194
Test Accuracy: 0.8133445978164673


In [ ]:
# SAVE THE MODEL
model.save('exercise_6_trained_model_improved.h5')

In [45]:
# SIMPLE INFERENCE SCRIPT
from tensorflow.keras.preprocessing import image

def predict_image(img_path, model_path='exercise_6_trained_model_improved.h5'):
    model = tf.keras.models.load_model(model_path)
    img = image.load_img(img_path, target_size=IMG_SIZE)
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    pred = model.predict(img_array)[0,0]
    label = "Muffin" if pred >= 0.5 else "Chihuahua"
    print(f"Prediction: {label} (confidence: {pred:.2f})")


In [46]:
# Example usage:
predict_image("run_1/run_1.jpg")
predict_image("run_1/run_2.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


Prediction: Chihuahua (confidence: 0.32)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Prediction: Muffin (confidence: 0.99)
